In [1]:
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q awscli

In [2]:
import os
os.environ['AWS_PROFILE'] = 'default'

In [3]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

1.37.28


In [4]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

In [5]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-west-2', '575108919340')

In [89]:
# configuration variables
suffix = f"{region}-{account_id}"
stack_name = "RadiologyReportValidator"
agent_name = f"{stack_name}-Agent"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}-{stack_name}"
agent_role_name = f'AmazonBRkExRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = "Agent for Validating Radiology Reports"
agent_instruction = "You are a Radiology Report Validator, helping junior radiologist \
                    write reports in adherence to the ACR guidance criterion. Does the radiology report adheres to the ACR guidelines mentioned in the document? \
                    Is it detailed enough to provide a diagnosis? \
                    Is the report missing any key anatomical structures? \
                    Does the report meet the \
                    quality standards of the ACR guidelines? Please provide a terse actionable feedback and do not try to summarize the report itself. ?"
agent_action_group_name = "RadiologyActionGroup"
agent_action_group_description = "Actions for Validating Radiology Reports or from Machine Learning Models"
agent_alias_name = f"{agent_name}-alias"


In [90]:
lambda_function_role = f"{stack_name}-LambdaExecutionRole"
lambda_function_name = f"{stack_name}-{account_id}-Lambda"
print(lambda_function_name)
lambda_function_arn = lambda_client.get_function(FunctionName=lambda_function_name)['Configuration']['FunctionArn']
# lambda_funtion_arn = "arn:aws:lambda:us-west-2:575108919340:function:RadiologyReportValidatorAgent-575108919340-Lambda"
# print(lambda_client.get_function(FunctionName=lambda_function_name))
lambda_client.get_function(FunctionName=lambda_function_name)["Configuration"]["FunctionArn"]

RadiologyReportValidator-575108919340-Lambda


'arn:aws:lambda:us-west-2:575108919340:function:RadiologyReportValidator-575108919340-Lambda'

In [102]:
# Upload the lambda function
# Package up the lambda function code and deploy to Lambda function
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda/lambda_function.py", arcname="lambda_function.py")
z.close()
zip_content = s.getvalue()

lambda_function = lambda_client.update_function_code(
    FunctionName=lambda_function_name,
    ZipFile=zip_content,
)

In [91]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)
try:
    agent_bedrock_policy = iam_client.create_policy(
        PolicyName=agent_bedrock_allow_policy_name,
        PolicyDocument=bedrock_policy_json
    )
except Exception:
    print(f"Policy {agent_bedrock_allow_policy_name} already exists")

Policy RadiologyReportValidator-Agent-ba-us-west-2-575108919340-RadiologyReportValidator already exists


In [17]:
print(agent_bedrock_policy)

{'Policy': {'PolicyName': 'RadiologyReportValidator-Agent-ba-us-west-2-575108919340-RadiologyReportValidator', 'PolicyId': 'ANPAYLZZJWAWFXMIOUESP', 'Arn': 'arn:aws:iam::575108919340:policy/RadiologyReportValidator-Agent-ba-us-west-2-575108919340-RadiologyReportValidator', 'Path': '/', 'DefaultVersionId': 'v1', 'AttachmentCount': 0, 'PermissionsBoundaryUsageCount': 0, 'IsAttachable': True, 'CreateDate': datetime.datetime(2025, 4, 6, 22, 6, 53, tzinfo=tzlocal()), 'UpdateDate': datetime.datetime(2025, 4, 6, 22, 6, 53, tzinfo=tzlocal())}, 'ResponseMetadata': {'RequestId': '1cfa2f38-5ea8-4f1c-8f96-11e208965c44', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 06 Apr 2025 22:06:53 GMT', 'x-amzn-requestid': '1cfa2f38-5ea8-4f1c-8f96-11e208965c44', 'content-type': 'text/xml', 'content-length': '897'}, 'RetryAttempts': 0}}


In [92]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
print(agent_bedrock_policy)
try:
    agent_role = iam_client.create_role(
        RoleName=agent_role_name,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )


    # Pause to make sure role is created
    time.sleep(10)
        
    iam_client.attach_role_policy(
        RoleName=agent_role_name,
        PolicyArn=agent_bedrock_policy['Policy']['Arn']
    )
except Exception as e:
    print(f"Error creating role: {e}. Make sure it doesnt exist already")

{'Policy': {'PolicyName': 'RadiologyReportValidator-Agent-ba-us-west-2-575108919340-RadiologyReportValidator', 'PolicyId': 'ANPAYLZZJWAWFXMIOUESP', 'Arn': 'arn:aws:iam::575108919340:policy/RadiologyReportValidator-Agent-ba-us-west-2-575108919340-RadiologyReportValidator', 'Path': '/', 'DefaultVersionId': 'v1', 'AttachmentCount': 0, 'PermissionsBoundaryUsageCount': 0, 'IsAttachable': True, 'CreateDate': datetime.datetime(2025, 4, 6, 22, 6, 53, tzinfo=tzlocal()), 'UpdateDate': datetime.datetime(2025, 4, 6, 22, 6, 53, tzinfo=tzlocal())}, 'ResponseMetadata': {'RequestId': '1cfa2f38-5ea8-4f1c-8f96-11e208965c44', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 06 Apr 2025 22:06:53 GMT', 'x-amzn-requestid': '1cfa2f38-5ea8-4f1c-8f96-11e208965c44', 'content-type': 'text/xml', 'content-length': '897'}, 'RetryAttempts': 0}}
Error creating role: An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name AmazonBRkExRoleForAgents_RadiologyReportValidator-Agent

In [93]:
try:
    response = bedrock_agent_client.create_agent(
        agentName=agent_name,
        agentResourceRoleArn=agent_role['Role']['Arn'],
        description=agent_description,
        idleSessionTTLInSeconds=1800,
        foundationModel=agent_foundation_model,
        instruction=agent_instruction,
    )
    agent_id = response['agent']['agentId']
    print(response)

except:
    print("Agent already exists, skipping creation")
    agent_info = [agent for agent in bedrock_agent_client.list_agents()['agentSummaries'] if agent['agentName']==agent_name][0]
    print(agent_info)
    agent_id = agent_info['agentId']
    agent_version = bedrock_agent_client.list_agent_versions(agentId=agent_id)['agentVersionSummaries'][0]['agentVersion']
    response = bedrock_agent_client.list_agent_action_groups(
        agentId=agent_id,
        agentVersion=agent_version
    )
    print(response['actionGroupSummaries'][0])
    action_group_id = response['actionGroupSummaries'][0]['actionGroupId']
    print(f"Using agent_id {agent_id} and action_group_id {action_group_id}")

Agent already exists, skipping creation
{'agentId': 'N8EIVIC9QU', 'agentName': 'RadiologyReportValidator-Agent', 'agentStatus': 'PREPARED', 'description': 'Agent for Validating Radiology Reports', 'updatedAt': datetime.datetime(2025, 4, 6, 22, 12, 5, 490240, tzinfo=tzlocal())}
{'actionGroupId': 'FDJLD9TCI2', 'actionGroupName': 'RadiologyActionGroup', 'actionGroupState': 'ENABLED', 'description': 'Actions for Validating Radiology Reports or from Machine Learning Models', 'updatedAt': datetime.datetime(2025, 4, 6, 22, 17, 25, 333082, tzinfo=tzlocal())}
Using agent_id N8EIVIC9QU and action_group_id FDJLD9TCI2


In [94]:
agent_functions = [
    {
        'name': 'validate_radiology_report',
        'description': 'validates the radiology report for a given report',
        'parameters': {
            "report": {
                "description": "The radiology report for a given patient",
                "required": True,
                "type": "string"
            }
        }
    }, 
    {
        'name' : 'check_radiology_report',
        'description': 'Check of the given text contains a radiology report', 
        'parameters': {
            'report': {
                'description': 'If the text is not a radiology report. Ask for the user to \
                provide the radiology report.',
                'required': True,
                'type': 'string'
            }
        }
    }, 
]

In [95]:
try:
    agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)
except:
    print("Action group already exists")
    agent_action_group_response = bedrock_agent_client.update_agent_action_group(
        agentId=agent_id,
        actionGroupId=action_group_id,
        agentVersion='DRAFT',
        actionGroupExecutor={
            'lambda': lambda_function_arn
        },
        actionGroupName=agent_action_group_name,
        functionSchema={
            'functions': agent_functions
        },
        description=agent_action_group_description
    )
    print(agent_action_group_response)

Action group already exists
{'ResponseMetadata': {'RequestId': '451fcbec-b9b0-4860-b0e3-77e88a37bb52', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 07 Apr 2025 02:26:04 GMT', 'content-type': 'application/json', 'content-length': '1124', 'connection': 'keep-alive', 'x-amzn-requestid': '451fcbec-b9b0-4860-b0e3-77e88a37bb52', 'x-amz-apigw-id': 'IoWlcEC2vHcEuUw=', 'x-amzn-trace-id': 'Root=1-67f337bc-0c0c59cd36ac9def7da25533'}, 'RetryAttempts': 0}, 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:575108919340:function:RadiologyReportValidator-575108919340-Lambda'}, 'actionGroupId': 'FDJLD9TCI2', 'actionGroupName': 'RadiologyActionGroup', 'actionGroupState': 'ENABLED', 'agentId': 'N8EIVIC9QU', 'agentVersion': 'DRAFT', 'clientToken': 'a4a6af36-1a1a-4445-9e89-3386770e5115', 'createdAt': datetime.datetime(2025, 4, 6, 22, 17, 25, 333082, tzinfo=tzlocal()), 'description': 'Actions for Validating Radiology Reports or from Machine Learning Models', 'functionS

In [96]:
try:
    response = lambda_client.add_permission(
        FunctionName=lambda_function_name,
        StatementId='allow_bedrock',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )
    print(response)
except: 
    print("Permission already exists")

Permission already exists


In [97]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

{'ResponseMetadata': {'RequestId': '4e0481ad-a3cb-43fb-9066-42e553b662ab', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Mon, 07 Apr 2025 02:26:13 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '4e0481ad-a3cb-43fb-9066-42e553b662ab', 'x-amz-apigw-id': 'IoWm2E4OPHcEEsQ=', 'x-amzn-trace-id': 'Root=1-67f337c5-3d5411502f6ca03f21624e51'}, 'RetryAttempts': 0}, 'agentId': 'N8EIVIC9QU', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2025, 4, 7, 2, 26, 13, 294732, tzinfo=tzlocal())}


In [107]:
test_query = "What is the best gene biomarker (lowest p value) with overall survival for patients that have undergone chemotherapy?"

x ="Here is the radiology report: FINAL REPORT EXAMINATION : CHEST ( PORTABLE AP ) INDICATION : History :___ F with ett placement TECHNIQUE : Upright AP view of the chest COMPARISON : None . Patient is currently listed as EU critical . \
FINDINGS : Endotracheal tube tip terminates approximately a 4.6 cm from the carina . Enteric tube tip terminates within the distal esophagus and should be advanced by at least 11 cm .Heart size is normal . Gaseous distension of the stomach is noted "

test_query = "Test"
session_id:str = str(uuid.uuid1())
print(agent_id)
# test_query = "Is this a good radiology report? "
response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query + x,
      agentId=agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)
answer=""
for event in response["completion"]:
    print(event.keys())
    print(event)
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

N8EIVIC9QU
dict_keys(['trace'])
{'trace': {'agentAliasId': 'TSTALIASID', 'agentId': 'N8EIVIC9QU', 'agentVersion': 'DRAFT', 'callerChain': [{'agentAliasArn': 'arn:aws:bedrock:us-west-2:575108919340:agent-alias/N8EIVIC9QU/TSTALIASID'}], 'eventTime': datetime.datetime(2025, 4, 7, 3, 0, 46, 68911, tzinfo=tzlocal()), 'sessionId': '803c6bbc-135c-11f0-89d2-febc3124094e', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 2048, 'stopSequences': ['</function_calls>', '</answer>', '</error>'], 'temperature': 0.0, 'topK': 250, 'topP': 1.0}, 'text': '{"system":" You are a Radiology Report Validator, helping junior radiologist write reports in adherence to the ACR guidance criterion. You have been provided with a set of functions to answer the user\'s question. You must call the functions in the format below: <function_calls>   <invoke>     <tool_name>$TOOL_NAME</tool_name>     <parameters>       <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME> 

In [106]:
print(answer)

Based on the provided radiology report for a chest x-ray, the key findings are:

1. The endotracheal tube tip is positioned approximately 4.6 cm from the carina, which appears to be an appropriate position.

2. The enteric tube tip is terminating in the distal esophagus and needs to be advanced by at least 11 cm to reach the stomach.

3. The heart size is normal.

4. There is gaseous distension of the stomach noted.

Overall, the report follows the standard format with the examination details, indication, technique, comparison, and findings sections. The findings are clearly described and actionable recommendations are provided regarding tube placements.
